<a href="https://colab.research.google.com/github/ppg18887/Demo1/blob/master/Multimodal_and_multi_task_version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autokeras
!pip install tensorflow==2.2.0rc2.

In [28]:
import os
import shutil

import numpy as np
import tensorflow as tf

import autokeras as ak

In [52]:
from tensorflow._api.v2.compat.v1 import int8
import numpy as np
import os
tf.config.run_functions_eagerly(True)
#from tensorflow.keras.preprocessing.image import img_to_array, load_img
import pandas as pd
#Loading images from google cloud
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"  # noqa: E501
local_file_path = tf.keras.utils.get_file(origin=dataset_url, fname="image_data", extract=True)

# The file is extracted in the same directory as the downloaded file.
local_dir_path = os.path.dirname(local_file_path)
# After check mannually, we know the extracted data is in 'flower_photos'.
data_dir = os.path.join(local_dir_path, "flower_photos")
print(data_dir)
#img = tf.keras.preprocessing.image.load_img(image_file)




#dividing images into training data and validation data

batch_size = 32
img_height = 180
img_width = 180

image_train_data = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
  )
#conversio to ndarray
for image_batch, labels_batch in image_train_data:
  train_image=image_batch.numpy()
  #print(train_image)
  print(train_image.shape)
  #print(labels_batch.shape)
  break


image_test_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)



/root/.keras/datasets/flower_photos
Found 3670 files belonging to 5 classes.
Using 2936 files for training.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


(32, 180, 180, 3)
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [55]:
structured_data = np.random.rand(100, 100).astype(np.float32)

In [51]:
from autokeras.tasks import structured_data
#Loading text data
dataset_url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

local_file_path = tf.keras.utils.get_file(
    fname="text_data",
    origin=dataset_url,
    extract=True,
)
# The file is extracted in the same directory as the downloaded file.
local_dir_path = os.path.dirname(local_file_path)
# After check mannually, we know the extracted data is in 'aclImdb'.
data_dir = os.path.join(local_dir_path, "aclImdb")
print(data_dir)

#Preparing training data and test data
structured_data = tf.keras.utils.text_dataset_from_directory(
    data_dir,
    batch_size=32,
    validation_split=0.2,
    subset='training',
    seed=42)
for text_batch, text_label in structured_data:
  train_text=text_batch.numpy()
  print(train_text)
  #print(labels_batch.shape)
  break

structured_test_data = tf.keras.utils.text_dataset_from_directory(
    os.path.join(data_dir, "test"), shuffle=False, batch_size=batch_size
)
for test_batch, test_label in structured_data:
  test_text=test_batch.numpy()
  print(type(test_text))
  print(test_text.shape)
  #print(labels_batch.shape)
  break

# Remove the unused data folder.

shutil.rmtree(os.path.join(data_dir, "train/unsup"))

/root/.keras/datasets/aclImdb
Found 100005 files belonging to 2 classes.
Using 80004 files for training.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


[b"I've tried to reconcile why so many bad reviews of this film, while the vast majority of reviews are given a rating of between 7 and 10. The reason may be this film is kind of hard to describe in a positive review, although a few have done that quite nicely already. This film is confusing, depressing, and doesn't have a happy ending. I still gave Pola X a rating of 10, because it is basically for me literature and art combined on film. That is really my favorite kind of filmmaking. I've only seen two of Carax's films: this one and Mauvis Sang. As with this film, I'm being somewhat pretentious when I call this one of Carax's best films- but I am. Carax has a minimalist style. If that type of film does not appeal to you and is boring, then it would be best not to watch this. But Pola X was less minimalist than Mauvis Sang, so it had quite a lot of intensity for a thriller- at least for my taste. I found it quite interesting and absorbing. The two lead roles did an excellent job acting

In [56]:
num_instances=100
# Generate regression targets.
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
# Generate classification labels of five classes.
classification_target = np.random.randint(5, size=num_instances)

In [57]:
# Initialize the multi with multiple inputs and outputs.
model = ak.AutoModel(
    inputs=[ak.ImageInput(), ak.StructuredDataInput()],
    outputs=[
        ak.RegressionHead(metrics=["mae"]),
        ak.ClassificationHead(loss="categorical_crossentropy", metrics=["accuracy"]),
    ],
    overwrite=True,
    max_trials=2,
)
# Fit the model with prepared data.
model.fit(
    [train_image, structured_data],
    [regression_target, classification_target],
    epochs=3,
)

ValueError: ignored